In [453]:
import requests
url = "http://localhost:8080"

game_url = f"{url}/game"
context_url = f"{url}/context"
agenda_url = f"{url}/agenda"
task_url = f"{url}/task"
overtime_url = f"{url}/overtime"
context_update_url = f"{url}/context-update"
explanation_url = f"{url}/explanation"
result_url = f"{url}/result"


In [454]:
def start_game(player_list: list[dict]):
    response = requests.post(
        game_url,
        json={"player_list": player_list}
    )
    
    return response.json()

def create_context(story: str, player_list: list[dict], max_turn: int):
    response = requests.post(
        context_url,
        json={"story": story, "player_list": player_list, "max_turn": max_turn}
    )
    
    return response.json()

def create_agenda(company_context: dict[str, str], player_context_list: list[dict[str, str]]):
    response = requests.post(
        agenda_url,
        json={"company_context": company_context, "player_context_list": player_context_list}
    )
    
    return response.json()

def create_task(company_context: dict, player_context_list: list[dict]):
    response = requests.post(
        task_url,
        json={"company_context": company_context, "player_context_list": player_context_list}
    )
    
    return response.json()

def create_overtime(company_context: dict, player_context_list: list[dict]):
    response = requests.post(
        overtime_url,
        json={"company_context": company_context, "player_context_list": player_context_list}
    )
    
    return response.json()

def update_context(company_context: dict, player_context_list: list[dict], agenda_list: list[dict], task_list: dict[str, list[dict]], overtime_task_list: dict[str, list[dict]]):
    response = requests.post(
        context_update_url,
        json={"company_context": company_context, "player_context_list": player_context_list, "agenda_list": agenda_list, "task_list": task_list, "overtime_task_list": overtime_task_list}
    )
    
    return response.json()

def create_explanation(company_context: dict, player_context_list: list[dict]):
    response = requests.post(
        explanation_url,
        json={"company_context": company_context, "player_context_list": player_context_list}
    )
    
    return response.json()

def calculate_result(company_context: dict, player_context_list: list[dict]):
    response = requests.post(
        result_url,
        json={"company_context": company_context, "player_context_list": player_context_list}
    )
    
    return response.json()

In [455]:
def story_stage(players):
    story_res = start_game(players)
    story = story_res["story"]
    
    print(story)
    return story

In [456]:
def context_stage(story, players, max_turn):
    context_res = create_context(story, players, max_turn)
    company_context = context_res["company_context"]
    player_context_list = context_res["player_context_list"]

    # # print company_context
    # for day, context in company_context.items():
    #     parsed_context = context.split(",")
        
    #     print(f"회사 상태:")
    #     for i in range(len(parsed_context)):
    #         print(f"{i+1}. {parsed_context[i]}")
    
    # for player in player_context_list:
    #     player_context = player["context"]        
    #     parsed_context = player_context.split(",")
        
    #     print(f"{player['name']}의 상태:")
    #     for i in range(len(parsed_context)):
    #         print(f"{i+1}. {parsed_context[i]}")

    return company_context, player_context_list

In [ ]:

def agenda_stage(company_context, player_context_list):
    agenda_res = create_agenda(company_context, player_context_list)
    agenda_list = agenda_res["agenda_list"]

    for agenda in agenda_list:
        print("--------------------------------")
        
        print(f"ID: {agenda['id']}")
        print(f"Name: {agenda['name']}")
        print(f"Description: {agenda['description']}")
        
        for i in range(len(agenda["agenda_options"])):
            print(f"Option {i}: {agenda['agenda_options'][i]['agenda_option_text']}")
        
        agenda["selected_options"] = {}
        for player in player_context_list:
            selected_option_idx = int(input(f"{player['name']}님, 선택지를 선택해주세요: "))
            agenda["selected_options"][player["id"]] = agenda["agenda_options"][selected_option_idx]
            
        print("--------------------------------")

    return agenda_list

In [458]:
def task_stage(company_context, player_context_list, players):
    task_res = create_task(company_context, player_context_list)
    task_list = task_res["task_list"]

    for player in players:
        player_id = player["id"]
        player_task = task_list[player_id]
        
        for task in player_task:
            print("--------------------------------")
            print(f"ID: {task['id']}")
            print(f"Name: {task['name']}")
            print(f"Description: {task['description']}")
            
            for i in range(len(task["options"])):
                print(f"Option {i}: {task['options'][i]['text']}")
                
            selected_option_idx = int(input("Select an option: "))
            task["selected_option"] = task["options"][selected_option_idx]
    
    return task_list


In [459]:
def overtime_stage(company_context, player_context_list, players):
    overtime_res = create_overtime(company_context, player_context_list)
    overtime_task_list = overtime_res["task_list"]
    
    for player in players:
        player_id = player["id"]
        player_overtime_task_list = overtime_task_list[player_id]
        
        for player_overtime_task in player_overtime_task_list:
            print("--------------------------------")
            
            print(f"ID: {player_overtime_task['id']}")
            print(f"Type: {player_overtime_task['type']}")
            print(f"Name: {player_overtime_task['name']}")
            print(f"Description: {player_overtime_task['description']}")
            
            for i in range(len(player_overtime_task["options"])):
                print(f"Option {i}: {player_overtime_task['options'][i]['text']}")
                
            selected_option_idx = int(input("Select an option: "))
            player_overtime_task["selected_option"] = player_overtime_task["options"][selected_option_idx]

    return overtime_task_list
            

In [460]:
def explanation_stage(company_context, player_context_list):
    explanation_res = create_explanation(company_context, player_context_list)
    explanation = explanation_res["explanation"]
    
    print(explanation)

In [461]:


def context_update_stage(company_context, player_context_list, agenda_list, task_list, overtime_task_list):
    try:
        context_res = update_context(
            company_context=company_context,
            player_context_list=player_context_list,
            agenda_list=agenda_list,
            task_list=task_list,
            overtime_task_list=overtime_task_list
        )
        
        company_context = context_res["company_context"]
        player_context_list = context_res["player_context_list"]
    except Exception as e:
        print(context_res)
        print(e)
    
    # explanation_stage(company_context, player_context_list)
    return company_context, player_context_list

In [462]:

players = [
    {
        "id": "1",
        "name": "John",
    },
    {
        "id": "2",
        "name": "Jane",
    },
    {
        "id": "3",
        "name": "Jim",
    },
    
]

max_turn = 3

story = story_stage(players)
company_context, player_context_list = context_stage(story, players, max_turn)

for _ in range(max_turn):
    explanation_stage(company_context, player_context_list)
    print("--------------------------------")
    
    # agenda_list = agenda_stage(company_context, player_context_list)
    
    # company_context, player_context_list = context_update_stage(company_context, player_context_list, agenda_list, {}, {})
    
    
    
    # task_list = task_stage(company_context, player_context_list, players)

    # company_context, player_context_list = context_update_stage(company_context, player_context_list, [], task_list, {})
    
    player_overtime_task_list = overtime_stage(company_context, player_context_list, players)
    
    company_context, player_context_list = context_update_stage(company_context, player_context_list, [], {}, player_overtime_task_list)
    
    print(company_context)
    print(player_context_list)
    
    print("--------------------------------")

result_res = calculate_result(company_context, player_context_list)


어두운 새벽, 낡은 사무실의 형광등 불빛 아래, 당신들은 서로를 마주보고 있습니다. 낡은 커피 머신에서는 씁쓸한 향이 피어오르고, 창밖으로는 아직 도시가 잠들어 있습니다. 이곳은 바로 '인사이트 허브', 당신들이 모든 것을 걸고 시작한 스타트업의 초라하지만 희망찬 보금자리입니다.

**John**, 당신은 이 모든 아이디어를 현실로 만들어낼 핵심 개발자입니다. 복잡한 코드와 시스템 아키텍처가 당신의 손끝에서 생명을 얻을 것입니다.

**Jane**, 당신은 사용자 경험의 마법사, 이 프로젝트의 얼굴을 책임질 디자이너입니다. 당신의 미적 감각과 통찰력으로 우리의 제품은 단순한 도구를 넘어 사용자들의 마음을 사로잡을 것입니다.

**Jim**, 당신은 이 배의 항해사, 전체 프로젝트의 방향을 제시하고 전략을 수립할 기획자입니다. 당신의 날카로운 분석과 비전 없이는 우리는 망망대해를 헤맬 뿐입니다.

오늘부터 당신들의 삶은 '데일리 스크럼'으로 시작될 것입니다. 짧지만 강렬한 회의를 통해 어제의 성과를 공유하고, 오늘의 목표를 설정하며, 서로의 어깨를 다독일 것입니다. 그리고 해가 뜨고 지는 동안, 당신들은 각자의 자리에서 미친 듯이 '일'할 것입니다. 때로는 밤샘 '야근'이 불가피할 것이고, 때로는 짧은 '휴식' 속에서 다음 도약을 위한 에너지를 충전해야 할 것입니다.

당신들의 눈앞에는 거대한 도전이 놓여 있습니다. 시장의 냉혹한 현실, 경쟁자들의 압박, 그리고 내부의 갈등까지. 하지만 당신들은 혼자가 아닙니다. 서로를 믿고, 서로의 재능을 존중하며, 이 작은 사무실에서 시작된 꿈을 현실로 만들어야 합니다. 당신들의 첫 번째 목표는 명확합니다: 이 프로젝트를 성공적으로 완수하고, '인사이트 허브'를 세상에 알리는 것입니다.

자, 이제 시작입니다. 당신들의 손에 이 스타트업의 미래가 달려 있습니다. 준비되었습니까?


JSONDecodeError: Expecting value: line 1 column 1 (char 0)